In [51]:
import cv2
from IPython.display import display
from ipywidgets import FloatProgress
import glob
import os
from shutil import rmtree
from subprocess import call
import time

# Augment data set

As described [here](http://docs.opencv.org/2.4/doc/user_guide/ug_traincascade.html#creating-training-set-from-a-single-image-and-a-collection-of-backgrounds-with-a-single-vec-file-as-an-output).

In [52]:
obj = 'robot'

In [ ]:
if os.name == 'posix':
    code_dirname = os.path.join(u'/home', 'baxter', 'ros_ws', 'src')
elif os.name == 'nt':
    code_dirname = os.path.join(u'\\\srv-file.brml.tum.de', 'nthome', 'mludersdorfer', 'reallyMyDocuments', 'code')
else:
    print 'wtf?'

package_dirname = os.path.join(code_dirname, 'baxter_pick_and_place')
print package_dirname

## Create cut-out base images from *_info.dat

Which was created with `opencv_1_dat_pos.ipynb`.

In [53]:
positive_dirname = os.path.join(package_dirname, 'data', 'sdd')

base_dirname = os.path.join(package_dirname, 
                            'data', 'synthetic_demo_data')
out_dirname = os.path.join(base_dirname, 'base', obj)
annotations_dirname = os.path.join(base_dirname, obj)
                           
if not os.path.exists(out_dirname):
    os.makedirs(out_dirname)
if os.path.exists(annotations_dirname):
    rmtree(annotations_dirname)
os.makedirs(annotations_dirname)

In [54]:
info_filename = os.path.join(positive_dirname, '%s_info.dat' % obj)

with open(info_filename, 'r') as fp:
    info = fp.read().splitlines()

for i in info:
    pth, _, roi = i.split('  ')
    img = cv2.imread(positive_dirname + pth)
    roi = [int(a) for a in roi.split(' ')]
    x, y, w, h = roi
    img = img[y:y+h, x:x+w]
    cv2.imwrite(os.path.join(out_dirname, pth.split('/')[-1]), img)
print "Wrote %i cut-out images into %s." % (len(info), out_dirname)

Wrote 25 cut-out images into /home/baxter/ros_ws/src/baxter_pick_and_place/data/synthetic_demo_data/base/robot.


## Augment cut-out images

In [55]:
base_images = glob.glob(os.path.join(out_dirname, '*.jpg'))
print "Found %i cut-out '%s'-samples." % (len(base_images), obj)

Found 25 cut-out 'robot'-samples.


In [56]:
num = 40  # number of augmentations per base-image
w = 20  # rescale augmented patches to width w
h = 20  # rescale augmented patches to height h

In [57]:
bg = os.path.join(base_dirname, 'bg.txt')

annotation_filenames = list()
f = FloatProgress(min=0, max=len(base_images))
f.description = "Augmenting '%s'-images" % obj
display(f)
start = time.time()
for idx, base_image in enumerate(base_images):
    info_dirname = os.path.join(annotations_dirname, str(idx).zfill(4))
    if not os.path.exists(info_dirname):
        os.makedirs(info_dirname)
    filename = '%s.lst' % os.path.splitext(base_image)[0].split('/')[-1]
    annotation_filenames.append(os.path.join(info_dirname, filename))
    
    cmd = 'opencv_createsamples -img %s -bg %s -info %s ' % \
        (base_image, 
         bg,
         annotation_filenames[-1]) + \
        '-num %i -w %i -h %i' % \
        (num,
         w,
         h) + \
        '-maxxangle 0.1 -maxyangle 0.1 -maxzangle 0.1 ' + \
        '-randinv -maxidev 255 -bgcolor 0 -bgthresh 0'
    call(cmd, shell=True)
    f.value = idx+1
print "Created %i samples from %i base images in %.2fs." % \
    (num*len(base_images), len(base_images), time.time()-start)

Created 1000 samples from 25 base images in 5.63s.


In [58]:
annotations_filename = os.path.join(base_dirname, '%s_info.dat' % obj)
print annotations_filename

/home/baxter/ros_ws/src/baxter_pick_and_place/data/synthetic_demo_data/robot_info.dat


In [59]:
with open(annotations_filename, 'w') as wfp:
    for af in annotation_filenames:
        pretext = os.path.join(*af.split('/')[-3:-1])
        with open(af, 'r') as rfp:
            for line in rfp:
                wfp.write(os.path.join(pretext, line))

## Combine annotations into *.vec

In [60]:
cmd = 'opencv_createsamples -vec %s -info %s -w %i -h %i -num %i' % \
    (os.path.join(base_dirname, '%s_%i_%i.vec' % (obj, w, h)), 
     annotations_filename, 
     w, 
     h, 
     num*len(annotation_filenames))
print '%s\n' % cmd
!{cmd}

opencv_createsamples -vec /home/baxter/ros_ws/src/baxter_pick_and_place/data/synthetic_demo_data/robot_20_20.vec -info /home/baxter/ros_ws/src/baxter_pick_and_place/data/synthetic_demo_data/robot_info.dat -w 20 -h 20 -num 1000

Info file name: /home/baxter/ros_ws/src/baxter_pick_and_place/data/synthetic_demo_data/robot_info.dat
Img file name: (NULL)
Vec file name: /home/baxter/ros_ws/src/baxter_pick_and_place/data/synthetic_demo_data/robot_20_20.vec
BG  file name: (NULL)
Num: 1000
BG color: 0
BG threshold: 80
Invert: FALSE
Max intensity deviation: 40
Max x angle: 1.1
Max y angle: 1.1
Max z angle: 0.5
Show samples: FALSE
Width: 20
Height: 20
Create training samples from images collection...
Done. Created 1000 samples
